In [16]:
import os
import shutil
import random
from pathlib import Path
import cv2

# 定义源数据文件夹路径
cf_folder = Path('./data/oct/training_set/train/CF')
oct_folder = Path('./data/oct/training_set/train/Cropped_OCT')

# 定义目标数据文件夹路径
output_folder = Path('./data/oct')

# 创建输出文件夹结构（0-5 文件夹，每个包含 train_A, train_B, test_A, test_B 子文件夹）
for i in range(6):
    for subset in ['train_A', 'train_B', 'test_A', 'test_B']:
        os.makedirs(output_folder / f'angel_{i}' / subset, exist_ok=True)

# 获取所有 CF 文件
cf_files = sorted(cf_folder.glob("*.jpg"))

# 设置训练和测试的比例
train_ratio = 0.8

resize_dim = (512, 512)

# 将 CF 文件按随机顺序划分为训练和测试集
for cf_file in cf_files:
    # 提取文件名 (如 0033L1 或 0033R1) 和对应的 OCT 文件夹
    filename = cf_file.stem  # e.g., 0033L1
    
    oct_subfolder = oct_folder / filename

    # 检查对应的 OCT 文件夹是否存在
    if not oct_subfolder.exists():
        print(f"OCT 文件夹 {oct_subfolder} 不存在，跳过...")
        continue
    
    # 随机划分为 train 或 test
    dataset_type = 'train' if random.random() < train_ratio else 'test'

    # 将 CF 文件复制到对应的文件夹
    for i in range(6):
        cf_dest_folder = output_folder / f"angel_{i}" / f"{dataset_type}_A"

        img_cf = cv2.imread(str(cf_file))
        img_cf_resized = cv2.resize(img_cf, resize_dim)

        cv2.imwrite(str(cf_dest_folder / cf_file.name), img_cf_resized)

    # 将 OCT 文件夹中的所有文件复制到对应的文件夹    
    for oct_image in oct_subfolder.glob("*.jpg"):
        angel_index = os.path.basename(oct_image).split('.')[0].split('_')[-1]
        oct_dest_folder = output_folder / f'angel_{angel_index}' / f"{dataset_type}_B"
        new_name = oct_image.name.replace(f'_{angel_index}', '')

        img_oct = cv2.imread(str(oct_image))
        img_oct_resized = cv2.resize(img_oct, resize_dim)
        
        cv2.imwrite(str(oct_dest_folder / new_name), img_oct_resized)

print("数据划分完成！")


数据划分完成！


In [1]:
import os
import json
from pathlib import Path

# 定义输出数据的主文件夹路径
output_folder = Path('./data/oct/')

# 遍历每个子文件夹（0, 1, 2, 3, 4, 5）
for i in range(6):
    subfolder = output_folder / f'angel_{i}'

    # 获取 train_CF 和 test_CF 文件夹路径
    train_cf_folder = subfolder / 'train_A'
    test_cf_folder = subfolder / 'test_A'

    # 为 train_prompts.json 生成文件内容
    train_prompts = {}
    for image_path in train_cf_folder.glob("*.jpg"):
        image_name = image_path.name  # 仅获取文件名
        if 'L.' in image_name:
            train_prompts[image_name] = "This is the color fundus image of left eye, please generate corresponding optical coherence tomography(OCT) image"
        else:
            train_prompts[image_name] = "This is the color fundus image of right eye, please generate corresponding optical coherence tomography(OCT) image"
        
    # 将 train_prompts 写入 train_prompts.json
    train_prompts_path = subfolder / 'train_prompts.json'
    with open(train_prompts_path, 'w') as f:
        json.dump(train_prompts, f, indent=2)
    
    # 为 test_prompts.json 生成文件内容
    test_prompts = {}
    for image_path in test_cf_folder.glob("*.jpg"):
        image_name = image_path.name  # 仅获取文件名
        if 'L.' in image_name:
            test_prompts[image_name] = "This is the color fundus image of left eye, please generate corresponding optical coherence tomography(OCT) image"
        else:
            test_prompts[image_name] = "This is the color fundus image of right eye, please generate corresponding optical coherence tomography(OCT) image"
        

    # 将 test_prompts 写入 test_prompts.json
    test_prompts_path = subfolder / 'test_prompts.json'
    with open(test_prompts_path, 'w') as f:
        json.dump(test_prompts, f, indent=2)

    print(f"子文件夹 {i} 的 train_prompts.json 和 test_prompts.json 已生成。")

print("所有 JSON 文件已生成！")


子文件夹 0 的 train_prompts.json 和 test_prompts.json 已生成。
子文件夹 1 的 train_prompts.json 和 test_prompts.json 已生成。
子文件夹 2 的 train_prompts.json 和 test_prompts.json 已生成。
子文件夹 3 的 train_prompts.json 和 test_prompts.json 已生成。
子文件夹 4 的 train_prompts.json 和 test_prompts.json 已生成。
子文件夹 5 的 train_prompts.json 和 test_prompts.json 已生成。
所有 JSON 文件已生成！
